In [ ]:
import os
import codecs
from random import randint
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image, ImageDraw
from data_generator import DataGenerator, Label
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# look for images and labels\
# !ls ../../data/images
# !ls ../../data/labels/training/label_2/

folder_with_images = "../../data/images/training/image_2/"
images_names = sorted(os.listdir(folder_with_images))
print("{} files from '{}' to '{}'".format(len(images_names), images_names[0], images_names[-1]))

folder_with_labels = "../../data/labels/training/label_2/"
labels_names = sorted(os.listdir(folder_with_labels))
print("{} files from '{}' to '{}'".format(len(labels_names), labels_names[0], labels_names[-1]))

# info about labels

values - name - description

   1 -   type   -      Describes the type of object: 'Car', 'Van', 'Truck',
                     'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram',
                     'Misc' or 'DontCare'
                     
   1  -  truncated -   Float from 0 (non-truncated) to 1 (truncated), where
                     truncated refers to the object leaving image boundaries
                     
   1 -   occluded   -  Integer (0,1,2,3) indicating occlusion state:
                     0 = fully visible, 1 = partly occluded
                     2 = largely occluded, 3 = unknown
                     
   1  -  alpha   -     Observation angle of object, ranging [-pi..pi]
   
   4  -  bbox     -    2D bounding box of object in the image (0-based index):
                     contains left, top, right, bottom pixel coordinates
                     
   3  -  dimensions -  3D object dimensions: height, width, length (in meters)
   
   3  -  location  -   3D object location x,y,z in camera coordinates (in meters)
   
   1  -  rotation_y  - Rotation ry around Y-axis in camera coordinates [-pi..pi]
   
   1  -  score     -   Only for results: Float, indicating confidence in
                     detection, needed for p/r curves, higher is better.

What's wrong:
1. Why are bbox coordinates float? 
2. occluded and truncated can be "-1"


# visualisation

In [ ]:
# look at first image
im = Image.open(folder_with_images + images_names[1])
print(im.format, im.size, im.mode)
plt.imshow(im)
plt.show()

In [ ]:
# look at first label
with codecs.open(folder_with_labels + labels_names[1]) as f:
    label_str = f.read()
    
print(label_str)
first_label = Label(label_str)

draw = ImageDraw.Draw(im)
draw.rectangle(((first_label.x_left, first_label.y_top), (first_label.x_right, first_label.y_bottom)), fill=None)
plt.imshow(im)
plt.show()
# im.show()

In [ ]:
def show_image(index:int, index_object=None)->None:
    im = Image.open(folder_with_images + images_names[index])
    draw = ImageDraw.Draw(im)  

    with codecs.open(folder_with_labels + labels_names[index]) as f:
        labels_str = f.read().strip()

    label_str = labels_str.split("\n")
    print(label_str)
    if index_object is not None:
        label = Label(label_str[index_object])
        draw.rectangle(((label.x_left, label.y_top), (label.x_right, label.y_bottom)), fill=None, outline = 'green')
    else:
        for label_str in label_str:
            label = Label(label_str)
            draw.rectangle(((label.x_left, label.y_top), (label.x_right, label.y_bottom)), fill=None)

#     plt.imshow(im)
#     plt.show()
    im.show()

In [ ]:
# look at random image abd label
image_num = randint(1, len(images_names)-1)

show_image(image_num)

 # compute statistics

In [ ]:
# What information do I want to see?
classes = []
widht = []
hight = []
occluded = []
name = []
index_in_file = []
truncated = []

In [ ]:
for label_name in labels_names:
    with codecs.open(folder_with_labels + label_name) as f:
        labels_str = f.read()[:-2]
    
    for i, label_str in enumerate(labels_str.split("\n")):
        label = Label(label_str)
        classes.append(label.object_type)
        occluded.append(label.occluded)
        widht.append(label.x_right-label.x_left)
        hight.append(label.y_bottom - label.y_top)
        name.append(label_name)
        index_in_file.append(i)
        truncated.append(label.truncated)

In [ ]:
stats = pd.DataFrame.from_records(zip(name, index_in_file, classes, widht, hight, occluded, truncated),
                               columns=["name", "index_in_file", "classes", "widht", "hight", "occluded", "truncated"])

In [ ]:
stats.head()

In [ ]:
print(stats.classes.value_counts())

In [ ]:
print(stats.occluded.value_counts())

In [ ]:
stats.describe()

In [ ]:
stats.widht.hist(bins=100)
plt.show()
print("min wight - {}, max wight - {}".format(min(widht), max(widht)))
stats.widht.hist(bins=100, range=[0, 50])
plt.show()


In [ ]:
stats[stats.widht<5]

there are a lot of strange labels with small widht and small heigth, what classifier will make error on. Ideally we should clean data and delete this objects from txt files. Also strange results for occluded == '-1'

In [ ]:
# show_image(4569, 3)

In [ ]:
print("{} - all objects; {} - all objects without DontCare, \
{} - without DontCare and truncated less then 70%".format(len(stats),
                                    len(stats[stats.classes != 'DontCare']),
                                    len(stats[(stats.classes != 'DontCare') & (stats.truncated < 0.7)])))

In [ ]:
stats[stats.occluded == -1].head()

In [ ]:
# Here I saw a lot of images with different truncated values.
show_image(1, 3)

# for choose window's size

In [ ]:
stats.widht.hist(bins=100)
# stats[(stats.classes != 'DontCare') & (stats.truncated < 0.7)].widht.hist(bins=100)
plt.show()

In [ ]:
stats.hight.hist(bins=100)
# stats[(stats.classes != 'DontCare') & (stats.truncated < 0.7)].hight.hist(bins=100)
plt.show()